In [1]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"

In [2]:
import os
import sys
import time
from dataclasses import dataclass, field
from pathlib import Path
from typing import Any, Callable, Dict, List, Optional, Union

import datasets
import numpy as np
from datasets import DatasetDict, load_dataset, load_metric
from tqdm import tqdm

import flax
import jax
import jax.numpy as jnp
import optax
import transformers
import wandb as wandb
from flax import core, jax_utils, struct, traverse_util
from flax.jax_utils import unreplicate
from flax.training.common_utils import get_metrics, shard, shard_prng_key
from huggingface_hub import Repository
from models.modeling_flax_wav2vec2 import FlaxWav2Vec2ForCTC
from models.configuration_wav2vec2 import Wav2Vec2Config
from optax._src import linear_algebra
from transformers import (
    AutoFeatureExtractor,
    AutoProcessor,
    AutoTokenizer,
    HfArgumentParser,
    TrainingArguments,
    is_tensorboard_available,
)
from transformers.file_utils import get_full_repo_name
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

/home/sanchitgandhi/hf/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1652441440.530639 1401483 tpu_initializer_helper.cc:116] libtpu.so is already in use by process with pid 1384520. Not attempting to load libtpu.so in this process.


In [3]:
@flax.struct.dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    feature_extractor_name: Optional[str] = field(
        default=None, metadata={"help": "feature extractor name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )
    freeze_feature_encoder: bool = field(
        default=True, metadata={"help": "Whether to freeze the feature encoder layers of the model."}
    )
    hidden_dropout: float = field(
        default=0.1,
        metadata={
            "help": "The dropout probability for all fully connected layers in the embeddings, encoder, and pooler."
        },
    )

In [4]:
@flax.struct.dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: str = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    text_column: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the column in the datasets containing the full texts (for summarization)."},
    )
    dataset_cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Path to cache directory for saving and loading datasets"}
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    max_test_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of test examples to this "
            "value if set."
        },
    )
    audio_column_name: str = field(
        default="audio",
        metadata={"help": "The name of the dataset column containing the audio data. Defaults to 'audio'"},
    )
    text_column_name: str = field(
        default="text",
        metadata={"help": "The name of the dataset column containing the text data. Defaults to 'text'"},
    )
    max_duration_in_seconds: float = field(
        default=20.0,
        metadata={
            "help": "Truncate audio files that are longer than `max_duration_in_seconds` seconds to 'max_duration_in_seconds`"
        },
    )
    min_duration_in_seconds: float = field(
        default=0.0, metadata={"help": "Filter audio files that are shorter than `min_duration_in_seconds` seconds"}
    )
    max_label_length: Optional[int] = field(
        default=512,
        metadata={
            "help": "The minimum total sequence length for target text after tokenization. Sequences shorter "
            "than this will be filtered."
        },
    )
    min_target_length: Optional[int] = field(
        default=0,
        metadata={
            "help": "The minimum total sequence length for target text after tokenization. Sequences shorter "
            "than this will be filtered."
        },
    )
    pad_input_to_multiple_of: Optional[int] = field(
        default=32000,
        metadata={
            "help": "If set will pad the input sequence to a multiple of the provided value. "
            "This is important to avoid triggering recompilations on TPU."
        },
    )
    pad_target_to_multiple_of: Optional[int] = field(
        default=None,
        metadata={
            "help": "If set will pad the target sequence to a multiple of the provided value. "
            "This is important to avoid triggering recompilations on TPU."
        },
    )
    preprocessing_only: bool = field(
        default=False,
        metadata={
            "help": "Whether to only do data preprocessing and skip training. "
            "This is especially useful when data preprocessing errors out in distributed training due to timeout. "
            "In this case, one should run the preprocessing in a non-distributed setup with `preprocessing_only=True` "
            "so that the cached datasets can consequently be loaded in distributed training"
        },
    )
    train_split_name: str = field(
        default="train",
        metadata={
            "help": "The name of the training data set split to use (via the datasets library). Defaults to 'train'"
        },
    )
    eval_split_name: str = field(
        default="test",
        metadata={
            "help": "The name of the training data set split to use (via the datasets library). Defaults to 'train'"
        },
    )
    do_lower_case: bool = field(
        default=True,
        metadata={"help": "Whether the target text should be lower cased."},
    )
    wandb_project: str = field(
        default="flax-speech-recognition-ctc",
        metadata={"help": "The name of the wandb project."},
    )
    wandb_name: str = field(
        default=None,
        metadata={"help": "The name of the wandb run."},
    )
    wandb_job_type: str = field(
        default="CTC",
        metadata={"help": "The name of the wandb job type."},
    )
    test_split_name: str = field(
        default="test",
        metadata={"help": "The name of the test data set split to use (via the datasets library). Defaults to 'test'"},
    )

In [5]:
@flax.struct.dataclass
class FlaxDataCollatorSpeechSeq2SeqWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor ([`Wav2Vec2Processor`])
            The processor used for proccessing the data.
        decoder_start_token_id (:obj: `int`)
            The begin-of-sentence of the decoder.
        input_padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned input sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        target_padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned target sequences (according to the model's padding side and padding index).
            See above for details.
        max_input_length (:obj:`float`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        pad_input_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the input sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
        pad_target_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the target sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Any
    input_padding: Union[bool, str] = "longest"
    label_padding: Union[bool, str] = "max_length"
    pad_input_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_label: Optional[int] = None
    max_input_length: Optional[float] = None
    max_label_length: Optional[float] = None

    def __call__(self, features: List[Dict[str, Union[List[int], np.ndarray]]]) -> Dict[str, np.ndarray]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # reformat list to dict and set to pytorch format
        batch = self.processor.feature_extractor.pad(
            input_features,
            max_length=self.max_input_length,
            padding=self.input_padding,
            pad_to_multiple_of=self.pad_input_to_multiple_of,
            return_tensors="np",
        )

        labels_batch = self.processor.tokenizer.pad(
            label_features,
            max_length=self.max_label_length,
            padding=self.label_padding,
            pad_to_multiple_of=self.pad_to_multiple_of_label,
            return_tensors="np",
        )

        labels = labels_batch["input_ids"]
        labels = np.ma.array(labels, mask=np.not_equal(labels_batch.attention_mask, 1))
        labels = labels.filled(fill_value=-100)

        batch["labels"] = labels

        return batch

In [6]:
def get_grouped_indices(
    dataset, batch_size: int, rng: Optional[List[int]] = None, mega_batch_mult: Optional[int] = None
) -> np.array:
    """
    Adapted from the `get_length_grouped_indices` function in the PyTorch Trainer utils file (https://github.com/huggingface/transformers/blob/main/src/transformers/trainer_pt_utils.py#L486)
    Function that returns a list of indices in which each slice of `batch_size` consecutive indices correspond to elements of similar
    lengths. To do this, the indices are:

    - randomly permuted (if a JAX rng is specified)
    - grouped in mega-batches of size `mega_batch_mult * batch_size`
    - sorted by length in each mega-batch

    The result is the concatenation of all mega-batches, with the batch of `batch_size` containing the element of
    maximum length placed first, so that an OOM happens sooner rather than later.
    """
    lengths = dataset["input_length"]

    # Default for mega_batch_mult: 50 or the number to get 4 megabatches, whichever is smaller.
    if mega_batch_mult is None:
        mega_batch_mult = min(len(lengths) // (batch_size * 4), 50)
        # Just in case, for tiny datasets
        if mega_batch_mult == 0:
            mega_batch_mult = 1

    # We need to use JAX for the random permutation as the PRNG key will be set based on the seed outside of the sampler.
    num_samples = len(lengths)
    indices = jax.random.permutation(rng, np.arange(num_samples)) if rng is not None else np.arange(num_samples)

    megabatch_size = mega_batch_mult * batch_size
    megabatches = [indices[i : i + megabatch_size].tolist() for i in range(0, len(lengths), megabatch_size)]
    megabatches = [list(sorted(megabatch, key=lambda i: lengths[i], reverse=True)) for megabatch in megabatches]

    # The rest is to get the biggest batch first.
    # Since each megabatch is sorted by descending length, the longest element is the first
    megabatch_maximums = [lengths[megabatch[0]] for megabatch in megabatches]
    max_idx = np.argmax(megabatch_maximums).item()
    # Switch to put the longest batch in first position
    # (note that this is different to the PT grouped sampler in which we only put the longest element in the first position, and not its batch)
    megabatches[0], megabatches[max_idx] = megabatches[max_idx], megabatches[0]

    megabatches = np.array([i for megabatch in megabatches for i in megabatch])

    return megabatches


def generate_batch_splits(samples_idx: np.ndarray, batch_size: int) -> np.ndarray:
    """Generate batches of data for a specified batch size from sample indices. If the dataset size is not divisible by
    the batch size, the last incomplete batch is dropped."""
    num_samples = len(samples_idx)
    samples_to_remove = num_samples % batch_size

    if samples_to_remove != 0:
        samples_idx = samples_idx[:-samples_to_remove]
    sections_split = num_samples // batch_size
    return samples_idx.reshape((sections_split, batch_size))

In [7]:
def ctc_loss(logits, logits_attention_mask, labels, blank_id, loss_reduction="mean", output_emission_dict=False, log_epsilon=-100000.0):
    """Computes CTC loss.
    This function performs forward computation over an FSA with `N * 2` states
    where `N` is the max number of labels. The states are split into two groups:
    Phi states and emission states. a phi-state accepts repetition of
    phi (blank)-symbols and transits to emission state when the correct label is
    observed. An emission state accepts repetition of the label and transits to
    the next phi states at any time (so called epsilon-transition).
    Below, `B` denotes the batch size, `T` denotes the time steps in `logits`,
    and `N` denotes the time steps in `labels`.
    Args:
      logits: (B, T, K)-array containing log-probabilities of each class.
      logitpaddings: (B, T)-array. Padding indicators for `logits`.
      labels: (B, N)-array containing reference integer labels.
      labelpaddings: (B, N)-array. Padding indicators for `labels`. Currently,
        `labels` must be right-padded, i.e. each row of `labelpaddings` must be
        repetition of zeroes, followed by repetition of ones.
      blank_id: Id for blank token.
      loss_reduction: one of "mean", "sum", "default"
        - "none": no reduction is applied.
        - "mean": output loss will be divided by target lengths and then the
          mean over the batch is taken.
        - "sum": output loss are summed over batch
      output_emission_dict: whether to output additional information about the emission probs
    Returns:
      A pair of `(per_seq_loss, aux)`.
      per_seq_loss:
        (B,)-array containing loss values for each sequence in the batch.
      aux: Dictionary containing interim variables used for computing losses.
        aux['logalpha_phi']: (T, B, N+1)-array. Log-forward-probabilities of each
          phi-state corresponding to the n-th label.
        aux['logalpha_emit']: (T, B, N)-array. Log-forward-probabilities of each
          emission-state corresponding to the n-th label.
        aux['logprobs_phi']: (T, B, 1)-array. Probability of the phi-symbol
          corresponding to each time frame.
        aux['logprobs_emit']: (T, B, N)-array. Probability of the n-th label
          corresponding to each time frame.
    """
    # label paddings are indicated by -100
    labelpaddings = labels < 0
    # logit paddings are the inverse of attention_mask
    logitpaddings = ~logits_attention_mask

    # Copied from https://github.com/tensorflow/lingvo/blob/master/lingvo/jax/layers/ctc_objectives.py
    batchsize, unused_maxinputlen, num_classes = logits.shape
    batchsize_, maxlabellen = labels.shape

    logprobs = jax.nn.log_softmax(logits)
    labellens = maxlabellen - jnp.sum(labelpaddings, axis=1).astype(jnp.int32)

    # repeat[b, n] == 1.0 when label[b, n] == label[b, n+1].
    repeat = (labels[:, :-1] == labels[:, 1:]).astype(jnp.float32)
    repeat = jnp.pad(repeat, ((0, 0), (0, 1)))

    logprobs_phi = logprobs[:, :, blank_id:blank_id + 1]  # [B, T, 1]
    logprobs_phi = jnp.transpose(logprobs_phi, (1, 0, 2))  # [T, B, 1]

    one_hot = jax.nn.one_hot(labels, num_classes=num_classes)  # [B, N, K]
    logprobs_emit = jnp.einsum('btk,bnk->btn', logprobs, one_hot)
    logprobs_emit = jnp.transpose(logprobs_emit, (1, 0, 2))  # [T, B, N]

    logalpha_phi_init = jnp.ones(
        (batchsize, maxlabellen + 1)) * log_epsilon  # [B, N]
    logalpha_phi_init = logalpha_phi_init.at[:, 0].set(0.0)
    logalpha_emit_init = jnp.ones(
        (batchsize, maxlabellen)) * log_epsilon  # [B, N]

    def loop_body(prev, x):
        prev_phi, prev_emit = prev
        # emit-to-phi epsilon transition, except if the next label is repetition
        prev_phi_orig = prev_phi
        prev_phi = prev_phi.at[:, 1:].set(
            jnp.logaddexp(prev_phi[:, 1:], prev_emit + log_epsilon * repeat))

        logprob_emit, logprob_phi, pad = x

        # phi-to-emit transition
        next_emit = jnp.logaddexp(prev_phi[:, :-1] + logprob_emit,
                                  prev_emit + logprob_emit)
        # self-loop transition
        next_phi = prev_phi + logprob_phi
        # emit-to-phi blank transition only when the next label is repetition
        next_phi = next_phi.at[:, 1:].set(jnp.logaddexp(next_phi[:, 1:], prev_emit + logprob_phi + log_epsilon * (1.0 - repeat)))

        pad = pad.reshape((batchsize, 1))
        next_emit = pad * prev_emit + (1.0 - pad) * next_emit
        next_phi = pad * prev_phi_orig + (1.0 - pad) * next_phi

        return (next_phi, next_emit), (next_phi, next_emit)

    xs = (logprobs_emit, logprobs_phi, logitpaddings.transpose((1, 0)))
    _, (logalpha_phi, logalpha_emit) = jax.lax.scan(loop_body, (logalpha_phi_init, logalpha_emit_init), xs)

    # last row needs to be updated with the last epsilon transition
    logalpha_phi_last = logalpha_phi[-1].at[:, 1:].set(
        jnp.logaddexp(logalpha_phi[-1, :, 1:], logalpha_emit[-1]))
    logalpha_phi = logalpha_phi.at[-1].set(logalpha_phi_last)

    # extract per_seq_loss
    one_hot = jax.nn.one_hot(labellens, num_classes=maxlabellen + 1)  # [B, N+1]
    per_seq_loss = -jnp.einsum('bn,bn->b', logalpha_phi_last, one_hot)

    if loss_reduction == "mean":
        target_lengths = labelpaddings.shape[-1] - labelpaddings.sum(axis=-1)
        loss = (per_seq_loss / target_lengths).mean()
    elif loss_reduction == "sum":
        loss = per_seq_loss.sum()
    else:
        loss = per_seq_loss

    if not output_emission_dict:
        return loss

    return loss, {
        'logalpha_phi': logalpha_phi,
        'logalpha_emit': logalpha_emit,
        'logprobs_phi': logprobs_phi,
        'logprobs_emit': logprobs_emit,
    }

In [8]:
model_args = ModelArguments(
    model_name_or_path="sanchit-gandhi/flax-wav2vec2-ctc-cv9-baseline-50k",
    tokenizer_name="patrickvonplaten/wav2vec2_ctc_cv9_tokenizer",
)

data_args = DataTrainingArguments(
    dataset_name="mozilla-foundation/common_voice_9_0",
    dataset_cache_dir="/home/sanchitgandhi/cache/huggingface/datasets/",
    dataset_config_name="en",
    train_split_name="train[:1%]",
    eval_split_name="validation",
    test_split_name="test",
    preprocessing_num_workers="1",
    text_column_name="sentence",
    max_duration_in_seconds=5,
)

training_args = TrainingArguments(per_device_train_batch_size=1, output_dir="./")

In [9]:
raw_datasets = DatasetDict()

raw_datasets["train"] = load_dataset(
            data_args.dataset_name,
            data_args.dataset_config_name,
            split=data_args.train_split_name,
            cache_dir=data_args.dataset_cache_dir,
            use_auth_token=True if model_args.use_auth_token else None,
        )

Using the latest cached version of the module from /home/sanchitgandhi/.cache/huggingface/modules/datasets_modules/datasets/mozilla-foundation--common_voice_9_0/26f54721b57ee2f31a333b315ed9151fbd8e693a3983c295fef63c67a12b9bf7 (last modified on Thu May  5 18:16:53 2022) since it couldn't be found locally at mozilla-foundation/common_voice_9_0., or remotely on the Hugging Face Hub.
Reusing dataset common_voice (/home/sanchitgandhi/cache/huggingface/datasets/mozilla-foundation___common_voice/en/9.0.0/26f54721b57ee2f31a333b315ed9151fbd8e693a3983c295fef63c67a12b9bf7)


In [10]:
config = Wav2Vec2Config.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    )

feature_extractor = AutoFeatureExtractor.from_pretrained(
        model_args.feature_extractor_name if model_args.feature_extractor_name else model_args.model_name_or_path,
    )
tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    )
    
model = FlaxWav2Vec2ForCTC.from_pretrained(
        model_args.model_name_or_path,
        config=config,
    )
processor = AutoProcessor.from_pretrained(model_args.model_name_or_path)

/home/sanchitgandhi/transformers/src/transformers/configuration_utils.py:358: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/home/sanchitgandhi/hf/lib/python3.8/site-packages/jax/_src/tree_util.py:188: FutureWarning: jax.tree_util.tree_multimap() is deprecated. Please use jax.tree_util.tree_map() instead as a drop-in replacement.
  warnings.warn('jax.tree_util.tree_multimap() is deprecated. Please use jax.tree_util.tree_map() '
tcmalloc: large alloc 1261895680 bytes == 0x61fe2000 @  0x7fb294716680 0x7fb294737824 0x5f8a01 0x648cf1 0x5c4676 0x4f290e 0x64f718 0x5048b3 0x56b1da 0x56939a 0x50aaa0 0x56c28c 0x56939a 0x68d047 0x6003a4 0x5c4a40 0x56b0ae 0x5002d8 0x56cadf 0x5002d8 0x56cadf 0x5002d8 0x503fb6 0x56b1da 0x5f6836 0x56b0ae 0x5f6836 0

In [11]:
# 6. Resample speech dataset ALWAYS
raw_datasets = raw_datasets.cast_column(
        data_args.audio_column_name, datasets.features.Audio(sampling_rate=feature_extractor.sampling_rate)
    )

In [12]:
# 7. Preprocessing the datasets.
# We need to read the audio files as arrays and tokenize the targets.
max_input_length = int(data_args.max_duration_in_seconds * feature_extractor.sampling_rate)
min_input_length = int(data_args.min_duration_in_seconds * feature_extractor.sampling_rate)
pad_input_to_multiple_of = data_args.pad_input_to_multiple_of
audio_column_name = data_args.audio_column_name
num_workers = int(data_args.preprocessing_num_workers)
text_column_name = data_args.text_column_name
model_input_name = feature_extractor.model_input_names[0]
do_lower_case = data_args.do_lower_case

def prepare_dataset(batch):
    # process audio
    sample = batch[audio_column_name]
    inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])
    # process audio length
    batch[model_input_name] = inputs.input_values[0]
    batch["input_length"] = len(batch["input_values"])

    # process targets
    input_str = batch[text_column_name].lower() if do_lower_case else batch[text_column_name]
    batch["labels"] = tokenizer(input_str).input_ids
    batch["labels_length"] = len(batch["labels"])
    return batch

vectorized_datasets = raw_datasets.map(
    prepare_dataset,
    remove_columns=next(iter(raw_datasets.values())).column_names,
    num_proc=num_workers,
    desc="preprocess dataset",
)

# filter data with inputs shorter than min_input_length or longer than max_input_length
def is_audio_in_length_range(length):
    return length > min_input_length and length < max_input_length

vectorized_datasets = vectorized_datasets.filter(
    is_audio_in_length_range,
    num_proc=num_workers,
    input_columns=["input_length"],
)

In [13]:
# 8. Load Metric
metric = load_metric("wer")

def compute_metrics(pred_ids: List[List[int]], label_ids: List[List[int]]):
    padded_ids = np.where(np.asarray(label_ids) == -100, tokenizer.pad_token_id, np.asarray(label_ids))

    pred_str = tokenizer.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = tokenizer.batch_decode(padded_ids, group_tokens=False)

    wer = metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}, pred_str, label_str

In [14]:
data_collator = FlaxDataCollatorSpeechSeq2SeqWithPadding(
        processor=processor,
        input_padding="longest",
        pad_input_to_multiple_of=pad_input_to_multiple_of,
        max_label_length=data_args.max_label_length,
    )

In [15]:
def create_learning_rate_fn(
    num_train_steps: int, num_warmup_steps: int, learning_rate: float
) -> Callable[[int], jnp.array]:
    """Returns a linear warmup, linear_decay learning rate function."""
    warmup_fn = optax.linear_schedule(init_value=0.0, end_value=learning_rate, transition_steps=num_warmup_steps)
    decay_fn = optax.linear_schedule(
        init_value=learning_rate, end_value=0, transition_steps=num_train_steps - num_warmup_steps
    )
    schedule_fn = optax.join_schedules(schedules=[warmup_fn, decay_fn], boundaries=[num_warmup_steps])
    return schedule_fn

linear_decay_lr_schedule_fn = create_learning_rate_fn(
        10,
        training_args.warmup_steps,
        training_args.learning_rate,
    )    

def decay_mask_fn(params):
    flat_params = traverse_util.flatten_dict(params)
    layer_norm_params = [
        (name, "scale")
        for name in ["layer_norm", "self_attn_layer_norm", "layernorm_embedding", "final_layer_norm"]
    ]
    flat_mask = {path: (path[-1] != "bias" and path[-2:] not in layer_norm_params) for path in flat_params}
    return traverse_util.unflatten_dict(flat_mask)

# Create AdamW optimizer
optim = optax.adamw(
    learning_rate=linear_decay_lr_schedule_fn,
    b1=training_args.adam_beta1,
    b2=training_args.adam_beta2,
    eps=training_args.adam_epsilon,
    weight_decay=training_args.weight_decay,
    mask=decay_mask_fn,
)

In [16]:
class CTCTrainState(struct.PyTreeNode):
    """Train state for use with a single Optax optimizer.
    Adapted from flax train_state https://github.com/google/flax/blob/main/flax/training/train_state.py
    """

    step: int
    apply_fn: Callable = struct.field(pytree_node=False)
    get_attention_mask_fn: Callable = struct.field(pytree_node=False)
    params: core.FrozenDict[str, Any]
    tx: optax.GradientTransformation = struct.field(pytree_node=False)
    opt_state: optax.OptState
    dropout_rng: jnp.ndarray
    max_grad_norm: Optional[float] = 1.0

    def apply_gradients(self, *, grads, **kwargs):
        """Updates `step`, `params`, `opt_state` and `**kwargs` in return value.

        Note that internally this function calls `.tx.update()` followed by a call
        to `optax.apply_updates()` to update `params` and `opt_state`.

        Args:
          grads: Gradients that have the same pytree structure as `.params`.
          **kwargs: Additional dataclass attributes that should be `.replace()`-ed.

        Returns:
          An updated instance of `self` with `step` incremented by one, `params`
          and `opt_state` updated by applying `grads`, and additional attributes
          replaced as specified by `kwargs`.
        """

        # clip gradients by global l2 norm
        g_norm = linear_algebra.global_norm(grads)
        g_norm = jnp.maximum(self.max_grad_norm, g_norm)
        grads = jax.tree_map(lambda t: (t / g_norm) * self.max_grad_norm, grads)

        updates, new_opt_state = self.tx.update(grads, self.opt_state, self.params)

        new_params = optax.apply_updates(self.params, updates)
        return self.replace(
            step=self.step + 1,
            params=new_params,
            opt_state=new_opt_state,
            **kwargs,
        )

    @classmethod
    def create(cls, *, apply_fn, params, tx, **kwargs):
        """Creates a new instance with `step=0` and initialized `opt_state`."""
        opt_state = tx.init(params)
        return cls(
            step=0,
            apply_fn=apply_fn,
            params=params,
            tx=tx,
            opt_state=opt_state,
            **kwargs,
        )

In [17]:
# 11. Initialize our training
rng = jax.random.PRNGKey(training_args.seed)
rng, dropout_rng = jax.random.split(rng)

In [18]:
# Setup train state
state = CTCTrainState.create(
    apply_fn=model.__call__,
    get_attention_mask_fn=model._get_feature_vector_attention_mask,
    params=model.params,
    tx=optim,
    dropout_rng=dropout_rng,
    max_grad_norm=training_args.max_grad_norm,
)

In [19]:
# Store some constants
max_steps = int(training_args.max_steps)
gradient_accumulation_steps = int(training_args.gradient_accumulation_steps)
train_batch_size = int(training_args.per_device_train_batch_size) * jax.device_count()
batch_size_per_update = train_batch_size * gradient_accumulation_steps
blank_id = model.config.pad_token_id

In [20]:
# Define gradient update step fn
def train_step(state, batch):
    # only one single rng per grad step, with or without accumulation, as the graph should be identical over one effective training batch
    dropout_rng, new_dropout_rng = jax.random.split(state.dropout_rng)

    def compute_loss(params, minibatch):
        labels = minibatch.pop("labels")
        logits = state.apply_fn(
            **minibatch,
            params=params,
            dropout_rng=dropout_rng,
            freeze_feature_encoder=model_args.freeze_feature_encoder,
            train=True,
        )[0]
        logits_mask = state.get_attention_mask_fn(logits.shape[1], batch["attention_mask"])
        loss = ctc_loss(logits, logits_mask, labels, blank_id, loss_reduction="mean")

        return loss

    #grad_fn = jax.value_and_grad(compute_loss)

    #loss, grad = grad_fn(state.params, batch)
    
    return compute_loss(state.params, batch)

In [21]:
# Generate an epoch by grouping by length
train_samples_idx = get_grouped_indices(vectorized_datasets["train"], batch_size_per_update)
train_batch_idx = generate_batch_splits(train_samples_idx, batch_size_per_update)[:5]

In [22]:
# Gather the indices for creating the batch and do a training step
for step, batch_idx in enumerate(tqdm(train_batch_idx, desc="Training...", position=1, leave=True), 1):
    samples = [vectorized_datasets["train"][int(idx)] for idx in batch_idx]
    batch = data_collator(samples)
    loss = train_step(state, batch)
    print(f"Step: {step} | Loss: {loss}")


Training...:  20%|██████████████▍                                                         | 1/5 [00:07<00:28,  7.22s/it]

Step: 1 | Loss: -0.6702111959457397



Training...:  40%|████████████████████████████▊                                           | 2/5 [00:09<00:13,  4.42s/it]

Step: 2 | Loss: -0.4611336290836334



Training...:  60%|███████████████████████████████████████████▏                            | 3/5 [00:13<00:07,  3.94s/it]

Step: 3 | Loss: -0.28208792209625244



Training...:  80%|█████████████████████████████████████████████████████████▌              | 4/5 [00:15<00:03,  3.37s/it]

Step: 4 | Loss: -0.23416008055210114



Training...: 100%|████████████████████████████████████████████████████████████████████████| 5/5 [00:17<00:00,  3.60s/it]

Step: 5 | Loss: -0.36222130060195923


In [36]:
# Redefine the CTC loss function but without any lax backend
def ctc_loss(logits, logits_attention_mask, labels, blank_id, loss_reduction="mean", output_emission_dict=False, log_epsilon=-100000.0):
    # label paddings are indicated by -100
    labelpaddings = labels < 0
    # logit paddings are the inverse of attention_mask
    logitpaddings = ~logits_attention_mask

    # Copied from https://github.com/tensorflow/lingvo/blob/master/lingvo/jax/layers/ctc_objectives.py
    batchsize, unused_maxinputlen, num_classes = logits.shape
    batchsize_, maxlabellen = labels.shape

    logprobs = jax.nn.log_softmax(logits)
    labellens = maxlabellen - jnp.sum(labelpaddings, axis=1).astype(jnp.int32)

    # repeat[b, n] == 1.0 when label[b, n] == label[b, n+1].
    repeat = (labels[:, :-1] == labels[:, 1:]).astype(jnp.float32)
    repeat = jnp.pad(repeat, ((0, 0), (0, 1)))

    logprobs_phi = logprobs[:, :, blank_id:blank_id + 1]  # [B, T, 1]
    logprobs_phi = jnp.transpose(logprobs_phi, (1, 0, 2))  # [T, B, 1]

    one_hot = jax.nn.one_hot(labels, num_classes=num_classes)  # [B, N, K]
    logprobs_emit = jnp.einsum('btk,bnk->btn', logprobs, one_hot)
    logprobs_emit = jnp.transpose(logprobs_emit, (1, 0, 2))  # [T, B, N]

    logalpha_phi_init = jnp.ones(
        (batchsize, maxlabellen + 1)) * log_epsilon  # [B, N]
    logalpha_phi_init = logalpha_phi_init.at[:, 0].set(0.0)
    logalpha_emit_init = jnp.ones(
        (batchsize, maxlabellen)) * log_epsilon  # [B, N]

    def loop_body(prev, x):
        prev_phi, prev_emit = prev
        # emit-to-phi epsilon transition, except if the next label is repetition
        prev_phi_orig = prev_phi
        prev_phi = prev_phi.at[:, 1:].set(
            jnp.logaddexp(prev_phi[:, 1:], prev_emit + log_epsilon * repeat))

        logprob_emit, logprob_phi, pad = x

        # phi-to-emit transition
        next_emit = jnp.logaddexp(prev_phi[:, :-1] + logprob_emit,
                                  prev_emit + logprob_emit)

        # self-loop transition
        next_phi = prev_phi + logprob_phi

        # emit-to-phi blank transition only when the next label is repetition
        next_phi = next_phi.at[:, 1:].set(jnp.logaddexp(next_phi[:, 1:], prev_emit + logprob_phi + log_epsilon * (1.0 - repeat)))

        pad = pad.reshape((batchsize, 1))
        next_emit = pad * prev_emit + (1.0 - pad) * next_emit
        next_phi = pad * prev_phi_orig + (1.0 - pad) * next_phi

        return (next_phi, next_emit), (next_phi, next_emit)

    xs = (logprobs_emit, logprobs_phi, logitpaddings.transpose((1, 0)))
    
    #_, (logalpha_phi, logalpha_emit) = jax.lax.scan(loop_body, (logalpha_phi_init, logalpha_emit_init), xs)
    
    
    # define our own scan funciton (no lax backend involvement)
    def scan(f, init, xs, length=None):
        if xs is None:
            xs = [None] * length
        carry = init
        ys = []
        for i in range(xs[0].shape[0]):
            carry, y = f(carry, (x[i] for x in xs))
            ys.append(y)
        y0 = jnp.stack([ys[i][0] for i in range(len(ys))])
        y1 = jnp.stack([ys[i][1] for i in range(len(ys))])
        return carry, (y0, y1)
    
    
    _, (logalpha_phi, logalpha_emit) = scan(loop_body, (logalpha_phi_init, logalpha_emit_init), xs)
    

    # last row needs to be updated with the last epsilon transition
    logalpha_phi_last = logalpha_phi[-1].at[:, 1:].set(
        jnp.logaddexp(logalpha_phi[-1, :, 1:], logalpha_emit[-1]))
    logalpha_phi = logalpha_phi.at[-1].set(logalpha_phi_last)

    # extract per_seq_loss
    one_hot = jax.nn.one_hot(labellens, num_classes=maxlabellen + 1)  # [B, N+1]
    per_seq_loss = -jnp.einsum('bn,bn->b', logalpha_phi_last, one_hot)

    if loss_reduction == "mean":
        target_lengths = labelpaddings.shape[-1] - labelpaddings.sum(axis=-1)
        loss = (per_seq_loss / target_lengths).mean()
    elif loss_reduction == "sum":
        loss = per_seq_loss.sum()
    else:
        loss = per_seq_loss

    if not output_emission_dict:
        return loss

    return loss, {
        'logalpha_phi': logalpha_phi,
        'logalpha_emit': logalpha_emit,
        'logprobs_phi': logprobs_phi,
        'logprobs_emit': logprobs_emit,
    }

In [37]:
# Gather the indices for creating the batch and do a training step -> check we get the same losses
for step, batch_idx in enumerate(tqdm(train_batch_idx, desc="Training...", position=1, leave=True), 1):
    samples = [vectorized_datasets["train"][int(idx)] for idx in batch_idx]
    batch = data_collator(samples)
    loss = train_step(state, batch)
    print(f"Step: {step} | Loss: {loss}")


Training...:  20%|██████████████▍                                                         | 1/5 [00:04<00:16,  4.19s/it]

Step: 1 | Loss: -0.6702111959457397



Training...:  40%|████████████████████████████▊                                           | 2/5 [00:08<00:12,  4.23s/it]

Step: 2 | Loss: -0.4611336290836334



Training...:  60%|███████████████████████████████████████████▏                            | 3/5 [00:12<00:08,  4.13s/it]

Step: 3 | Loss: -0.28208792209625244



Training...:  80%|█████████████████████████████████████████████████████████▌              | 4/5 [00:16<00:04,  4.20s/it]

Step: 4 | Loss: -0.23416008055210114



Training...: 100%|████████████████████████████████████████████████████████████████████████| 5/5 [00:20<00:00,  4.18s/it]

Step: 5 | Loss: -0.36222130060195923


In [39]:
# Add some print statements in the 'loop_body' function
def ctc_loss(logits, logits_attention_mask, labels, blank_id, loss_reduction="mean", output_emission_dict=False, log_epsilon=-100000.0):
    # label paddings are indicated by -100
    labelpaddings = labels < 0
    # logit paddings are the inverse of attention_mask
    logitpaddings = ~logits_attention_mask

    # Copied from https://github.com/tensorflow/lingvo/blob/master/lingvo/jax/layers/ctc_objectives.py
    batchsize, unused_maxinputlen, num_classes = logits.shape
    batchsize_, maxlabellen = labels.shape

    logprobs = jax.nn.log_softmax(logits)
    labellens = maxlabellen - jnp.sum(labelpaddings, axis=1).astype(jnp.int32)

    # repeat[b, n] == 1.0 when label[b, n] == label[b, n+1].
    repeat = (labels[:, :-1] == labels[:, 1:]).astype(jnp.float32)
    repeat = jnp.pad(repeat, ((0, 0), (0, 1)))

    logprobs_phi = logprobs[:, :, blank_id:blank_id + 1]  # [B, T, 1]
    logprobs_phi = jnp.transpose(logprobs_phi, (1, 0, 2))  # [T, B, 1]

    one_hot = jax.nn.one_hot(labels, num_classes=num_classes)  # [B, N, K]
    logprobs_emit = jnp.einsum('btk,bnk->btn', logprobs, one_hot)
    logprobs_emit = jnp.transpose(logprobs_emit, (1, 0, 2))  # [T, B, N]

    logalpha_phi_init = jnp.ones(
        (batchsize, maxlabellen + 1)) * log_epsilon  # [B, N]
    logalpha_phi_init = logalpha_phi_init.at[:, 0].set(0.0)
    logalpha_emit_init = jnp.ones(
        (batchsize, maxlabellen)) * log_epsilon  # [B, N]

    def loop_body(prev, x):
        prev_phi, prev_emit = prev
        # emit-to-phi epsilon transition, except if the next label is repetition
        prev_phi_orig = prev_phi
        prev_phi = prev_phi.at[:, 1:].set(
            jnp.logaddexp(prev_phi[:, 1:], prev_emit + log_epsilon * repeat))
        if (prev_phi > 0).any():
            print("Emit-to-phi, prev phi: ", prev_phi[prev_phi > 0])

        logprob_emit, logprob_phi, pad = x

        # phi-to-emit transition
        next_emit = jnp.logaddexp(prev_phi[:, :-1] + logprob_emit,
                                  prev_emit + logprob_emit)

        # check for positive values
        if (next_emit > 0).any():
            print("Phi-to-emit, next emit: ", next_emit[next_emit > 0])

        # self-loop transition
        next_phi = prev_phi + logprob_phi
        if (next_phi > 0).any():
            print("Self-loop, next phi: ", next_phi[next_phi > 0])

        # emit-to-phi blank transition only when the next label is repetition
        next_phi = next_phi.at[:, 1:].set(jnp.logaddexp(next_phi[:, 1:], prev_emit + logprob_phi + log_epsilon * (1.0 - repeat)))
        if (next_phi > 0).any():
            print("Emit-to-phi blank, next phi: ", next_phi[next_phi > 0])

        pad = pad.reshape((batchsize, 1))
        next_emit = pad * prev_emit + (1.0 - pad) * next_emit
        next_phi = pad * prev_phi_orig + (1.0 - pad) * next_phi

        # next_emit = jnp.where(next_emit > 0, 0, next_emit)
        # next_phi = jnp.where(next_phi > 0, 0, next_phi)

        return (next_phi, next_emit), (next_phi, next_emit)

    xs = (logprobs_emit, logprobs_phi, logitpaddings.transpose((1, 0)))
    
    #_, (logalpha_phi, logalpha_emit) = jax.lax.scan(loop_body, (logalpha_phi_init, logalpha_emit_init), xs)
    
    
    # define our own scan funciton (no lax backend involvement)
    def scan(f, init, xs, length=None):
        if xs is None:
            xs = [None] * length
        carry = init
        ys = []
        for i in range(xs[0].shape[0]):
            print("Loop ", i)
            carry, y = f(carry, (x[i] for x in xs))
            ys.append(y)
        y0 = jnp.stack([ys[i][0] for i in range(len(ys))])
        y1 = jnp.stack([ys[i][1] for i in range(len(ys))])
        return carry, (y0, y1)
    
    
    _, (logalpha_phi, logalpha_emit) = scan(loop_body, (logalpha_phi_init, logalpha_emit_init), xs)
    

    # last row needs to be updated with the last epsilon transition
    logalpha_phi_last = logalpha_phi[-1].at[:, 1:].set(
        jnp.logaddexp(logalpha_phi[-1, :, 1:], logalpha_emit[-1]))
    logalpha_phi = logalpha_phi.at[-1].set(logalpha_phi_last)

    # extract per_seq_loss
    one_hot = jax.nn.one_hot(labellens, num_classes=maxlabellen + 1)  # [B, N+1]
    per_seq_loss = -jnp.einsum('bn,bn->b', logalpha_phi_last, one_hot)

    if loss_reduction == "mean":
        target_lengths = labelpaddings.shape[-1] - labelpaddings.sum(axis=-1)
        loss = (per_seq_loss / target_lengths).mean()
    elif loss_reduction == "sum":
        loss = per_seq_loss.sum()
    else:
        loss = per_seq_loss

    if not output_emission_dict:
        return loss

    return loss, {
        'logalpha_phi': logalpha_phi,
        'logalpha_emit': logalpha_emit,
        'logprobs_phi': logprobs_phi,
        'logprobs_emit': logprobs_emit,
    }

In [40]:
# Gather the indices for creating the batch and do a training step
for step, batch_idx in enumerate(tqdm(train_batch_idx[:1], desc="Training...", position=1, leave=True), 1):
    samples = [vectorized_datasets["train"][int(idx)] for idx in batch_idx]
    batch = data_collator(samples)
    loss = train_step(state, batch)
    print(f"Step: {step} | Loss: {loss}")


Training...:   0%|                                                                                | 0/1 [00:00<?, ?it/s]

Loop  0
Loop  1
Loop  2
Loop  3
Loop  4
Phi-to-emit, next emit:  [0.6905685]
Loop  5
Emit-to-phi, prev phi:  [1.0959988]
Phi-to-emit, next emit:  [1.0959431]
Self-loop, next phi:  [1.0958912]
Emit-to-phi blank, next phi:  [1.0958912]
Loop  6
Emit-to-phi, prev phi:  [1.7890644]
Phi-to-emit, next emit:  [1.383553]
Self-loop, next phi:  [1.7889559]
Emit-to-phi blank, next phi:  [1.7889559]
Loop  7
Emit-to-phi, prev phi:  [2.2998064]
Phi-to-emit, next emit:  [1.6066316]
Self-loop, next phi:  [2.2998025]
Emit-to-phi blank, next phi:  [2.2998025]
Loop  8
Emit-to-phi, prev phi:  [2.7052598]
Phi-to-emit, next emit:  [1.7889092]
Self-loop, next phi:  [2.7052555]
Emit-to-phi blank, next phi:  [2.7052555]
Loop  9
Emit-to-phi, prev phi:  [3.0417118]
Phi-to-emit, next emit:  [1.9430279]
Self-loop, next phi:  [3.041684]
Emit-to-phi blank, next phi:  [3.041684]
Loop  10
Emit-to-phi, prev phi:  [3.329355]
Phi-to-emit, next emit:  [2.0765316]
Self-loop, next phi:  [3.329328]
Emit-to-phi blank, next phi


Training...: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.76s/it]

Emit-to-phi blank, next phi:  [ 4.841214  10.903507   3.9163907 15.207778   4.2398977 18.81793
  8.518696  23.244719  16.257605  10.338607   1.4322771 28.710772
 18.370634   5.047669   6.284687  14.055418  20.262394  28.238472
 30.129286  30.100508  25.412271  22.996029  10.019483  11.362205
 12.957952  14.182192  15.071685  15.653526  15.945742  15.957383
 15.689164  15.134707  14.28189   13.113819  11.609209   9.742163
  7.4810705  4.7856865  1.599571 ]
Loop  298
Emit-to-phi, prev phi:  [ 4.8714323  10.933725    3.9466085  15.237996    4.270116   18.848148
  8.548914   23.274937   16.287823   10.368825    1.462495   28.74099
 18.400852    5.077887    6.314905   14.085636   20.292612   28.26869
 30.159504   25.971088   24.443487   21.58461     8.690807   10.6628895
 12.22039    13.411439   14.271389   14.825957   15.09196    15.077371
 14.781918   14.198311   13.313482   12.109287   10.562489    8.644138
  6.3184204   3.5406983   0.25335878]
Phi-to-emit, next emit:  [ 4.0079637  10.07